In [9]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')
import tensorflow as tf

import math
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import pickle
import gzip
import tensorflow as tf

# Data Load

In [10]:
label = np.load("./data/2030days_label.npy")
data_10days = np.load("./data/10days_data.npy")
data_20days = np.load("./data/20days_data.npy")
data_30days = np.load("./data/30days_data.npy")

label.shape, data_10days.shape, data_20days.shape, data_30days.shape

((7799,), (7799, 10, 4069), (7799, 20, 4069), (7799, 30, 4069))

In [11]:
def MAE(y_true, y_pred):
    return "{:.2e}".format(np.mean(np.abs(y_true-y_pred)))

def MSE(y_true, y_pred):
    return "{:.2e}".format(np.mean(np.square(y_true-y_pred)))

def RMSE(y_true, y_pred):
    return "{:.2e}".format(np.sqrt(np.mean(np.square(y_true-y_pred))))

In [12]:
from sklearn.ensemble import AdaBoostRegressor, AdaBoostClassifier
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from sklearn.ensemble import VotingClassifier
from keras.layers import InputLayer, GRU

In [13]:
# 사용할 data 지정
data = data_10days

#train, test split
i = round(data.shape[0] * 0.8)
X_train = data[:i,:,:]
y_train = label[:i]


X_test = data[i:,:,:]
y_test = label[i:]


X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6239, 10, 4069), (6239,), (1560, 10, 4069), (1560,))

# Modeling

In [14]:
tf.random.set_seed(1004)

In [15]:
%%time
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(units=128, return_sequences=True))
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=32, return_sequences=False))
# model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])
model.summary()

estimator = []
for i in range(1,51):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:model, epochs=20, batch_size=256)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print("RMSE : ", RMSE(y_test, preds))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 128)           2149376   
_________________________________________________________________
lstm_4 (LSTM)                (None, 10, 64)            49408     
_________________________________________________________________
lstm_5 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 2,211,233
Trainable params: 2,211,233
Non-trainable params: 0
_________________________________________________________________
[('model1', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f272c4b01c0>), ('model2', <keras.wrapper

25/25 [==============================] - 6s 242ms/step - loss: 0.0053 - acc: 0.9968
Epoch 17/20
25/25 [==============================] - 6s 221ms/step - loss: 0.0050 - acc: 0.9968
Epoch 18/20
25/25 [==============================] - 6s 222ms/step - loss: 0.0050 - acc: 0.9968
Epoch 19/20
25/25 [==============================] - 6s 230ms/step - loss: 0.0054 - acc: 0.9965
Epoch 20/20
25/25 [==============================] - 6s 234ms/step - loss: 0.0053 - acc: 0.9965
Epoch 1/20
25/25 [==============================] - 6s 237ms/step - loss: 0.0062 - acc: 0.9955
Epoch 2/20
25/25 [==============================] - 6s 236ms/step - loss: 0.0077 - acc: 0.9934
Epoch 3/20
25/25 [==============================] - 6s 239ms/step - loss: 0.0110 - acc: 0.9889
Epoch 4/20
25/25 [==============================] - 6s 224ms/step - loss: 0.0155 - acc: 0.9840
Epoch 5/20
25/25 [==============================] - 6s 232ms/step - loss: 0.0165 - acc: 0.9832
Epoch 6/20
25/25 [==============================] - 6s 24

25/25 [==============================] - 5s 202ms/step - loss: 0.0034 - acc: 0.9981
Epoch 7/20
25/25 [==============================] - 5s 207ms/step - loss: 0.0034 - acc: 0.9981
Epoch 8/20
25/25 [==============================] - 5s 208ms/step - loss: 0.0034 - acc: 0.9981
Epoch 9/20
25/25 [==============================] - 6s 236ms/step - loss: 0.0034 - acc: 0.9981
Epoch 10/20
25/25 [==============================] - 6s 245ms/step - loss: 0.0033 - acc: 0.9981
Epoch 11/20
25/25 [==============================] - 6s 251ms/step - loss: 0.0034 - acc: 0.9981
Epoch 12/20
25/25 [==============================] - 6s 243ms/step - loss: 0.0035 - acc: 0.9981
Epoch 13/20
25/25 [==============================] - 6s 250ms/step - loss: 0.0034 - acc: 0.9981
Epoch 14/20
25/25 [==============================] - 6s 246ms/step - loss: 0.0035 - acc: 0.9981
Epoch 15/20
25/25 [==============================] - 6s 247ms/step - loss: 0.0035 - acc: 0.9981
Epoch 16/20
25/25 [==============================] - 5s

25/25 [==============================] - 6s 251ms/step - loss: 0.0034 - acc: 0.9981
Epoch 17/20
25/25 [==============================] - 6s 230ms/step - loss: 0.0034 - acc: 0.9981
Epoch 18/20
25/25 [==============================] - 6s 237ms/step - loss: 0.0034 - acc: 0.9981
Epoch 19/20
25/25 [==============================] - 6s 225ms/step - loss: 0.0034 - acc: 0.9981
Epoch 20/20
25/25 [==============================] - 6s 234ms/step - loss: 0.0034 - acc: 0.9981
Epoch 1/20
25/25 [==============================] - 6s 243ms/step - loss: 0.0033 - acc: 0.9981
Epoch 2/20
25/25 [==============================] - 6s 246ms/step - loss: 0.0033 - acc: 0.9981
Epoch 3/20
25/25 [==============================] - 6s 243ms/step - loss: 0.0034 - acc: 0.9981
Epoch 4/20
25/25 [==============================] - 6s 233ms/step - loss: 0.0034 - acc: 0.9981
Epoch 5/20
25/25 [==============================] - 6s 247ms/step - loss: 0.0034 - acc: 0.9981
Epoch 6/20
25/25 [==============================] - 6s 25

25/25 [==============================] - 5s 208ms/step - loss: 0.0034 - acc: 0.9981
Epoch 7/20
25/25 [==============================] - 5s 209ms/step - loss: 0.0033 - acc: 0.9981
Epoch 8/20
25/25 [==============================] - 5s 211ms/step - loss: 0.0035 - acc: 0.9981
Epoch 9/20
25/25 [==============================] - 5s 203ms/step - loss: 0.0033 - acc: 0.9981
Epoch 10/20
25/25 [==============================] - 5s 204ms/step - loss: 0.0033 - acc: 0.9981
Epoch 11/20
25/25 [==============================] - 5s 207ms/step - loss: 0.0034 - acc: 0.9981
Epoch 12/20
25/25 [==============================] - 5s 208ms/step - loss: 0.0034 - acc: 0.9981
Epoch 13/20
25/25 [==============================] - 5s 196ms/step - loss: 0.0034 - acc: 0.9981
Epoch 14/20
25/25 [==============================] - 4s 151ms/step - loss: 0.0034 - acc: 0.9981
Epoch 15/20
25/25 [==============================] - 4s 177ms/step - loss: 0.0033 - acc: 0.9981
Epoch 16/20
25/25 [==============================] - 4s

25/25 [==============================] - 2s 73ms/step - loss: 0.0036 - acc: 0.9976
Epoch 18/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0039 - acc: 0.9973
Epoch 19/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0034 - acc: 0.9981
Epoch 20/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0034 - acc: 0.9981
Epoch 1/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0034 - acc: 0.9981
Epoch 2/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0036 - acc: 0.9979
Epoch 3/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0037 - acc: 0.9978
Epoch 4/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0034 - acc: 0.9979
Epoch 5/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0034 - acc: 0.9981
Epoch 6/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0033 - acc: 0.9981
Epoch 7/20
25/25 [==============================] - 2s 75ms/step - l

25/25 [==============================] - 2s 75ms/step - loss: 0.0034 - acc: 0.9981
Epoch 10/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0034 - acc: 0.9981
Epoch 11/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0034 - acc: 0.9981
Epoch 12/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0034 - acc: 0.9981
Epoch 13/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0034 - acc: 0.9981
Epoch 14/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0033 - acc: 0.9981
Epoch 15/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0034 - acc: 0.9981
Epoch 16/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0033 - acc: 0.9981
Epoch 17/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0033 - acc: 0.9981
Epoch 18/20
25/25 [==============================] - 2s 74ms/step - loss: 0.0033 - acc: 0.9981
Epoch 19/20
25/25 [==============================] - 2s 72ms/s

In [9]:
import time
start = time.time()  # 시작 시간 저장
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(units=128, return_sequences=True))
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=32, return_sequences=False))
# model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])
model.summary()

estimator = []
for i in range(1,51):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:model, epochs=100, batch_size=1024)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print("time :", (time.time() - start)/60, '분') 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 128)           2149376   
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 64)            49408     
_________________________________________________________________
lstm_2 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout (Dropout)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,211,233
Trainable params: 2,211,233
Non-trainable params: 0
_________________________________________________________________
[('model1', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7f358c7eeaf0>), ('model2', <keras.wrappers.

Epoch 32/100
7/7 [==============================] - 2s 326ms/step - loss: 0.0323 - acc: 0.9681
Epoch 33/100
7/7 [==============================] - 2s 329ms/step - loss: 0.0333 - acc: 0.9643
Epoch 34/100
7/7 [==============================] - 2s 322ms/step - loss: 0.0285 - acc: 0.9737
Epoch 35/100
7/7 [==============================] - 2s 321ms/step - loss: 0.0276 - acc: 0.9756
Epoch 36/100
7/7 [==============================] - 2s 348ms/step - loss: 0.0269 - acc: 0.9743
Epoch 37/100
7/7 [==============================] - 2s 316ms/step - loss: 0.0274 - acc: 0.9752
Epoch 38/100
7/7 [==============================] - 2s 332ms/step - loss: 0.0243 - acc: 0.9775
Epoch 39/100
7/7 [==============================] - 2s 335ms/step - loss: 0.0236 - acc: 0.9786
Epoch 40/100
7/7 [==============================] - 2s 339ms/step - loss: 0.0244 - acc: 0.9789
Epoch 41/100
7/7 [==============================] - 2s 353ms/step - loss: 0.0235 - acc: 0.9781
Epoch 42/100
7/7 [==============================] 

7/7 [==============================] - 2s 320ms/step - loss: 0.0070 - acc: 0.9962
Epoch 5/100
7/7 [==============================] - 2s 330ms/step - loss: 0.0070 - acc: 0.9962
Epoch 6/100
7/7 [==============================] - 2s 331ms/step - loss: 0.0068 - acc: 0.9963
Epoch 7/100
7/7 [==============================] - 2s 330ms/step - loss: 0.0068 - acc: 0.9958
Epoch 8/100
7/7 [==============================] - 2s 348ms/step - loss: 0.0069 - acc: 0.9962
Epoch 9/100
7/7 [==============================] - 2s 338ms/step - loss: 0.0069 - acc: 0.9963
Epoch 10/100
7/7 [==============================] - 2s 336ms/step - loss: 0.0069 - acc: 0.9965
Epoch 11/100
7/7 [==============================] - 2s 338ms/step - loss: 0.0067 - acc: 0.9965
Epoch 12/100
7/7 [==============================] - 2s 341ms/step - loss: 0.0066 - acc: 0.9966
Epoch 13/100
7/7 [==============================] - 2s 328ms/step - loss: 0.0065 - acc: 0.9965
Epoch 14/100
7/7 [==============================] - 2s 340ms/step - 

7/7 [==============================] - 2s 324ms/step - loss: 0.0047 - acc: 0.9971
Epoch 77/100
7/7 [==============================] - 2s 339ms/step - loss: 0.0046 - acc: 0.9974
Epoch 78/100
7/7 [==============================] - 2s 330ms/step - loss: 0.0046 - acc: 0.9973
Epoch 79/100
7/7 [==============================] - 2s 353ms/step - loss: 0.0046 - acc: 0.9974
Epoch 80/100
7/7 [==============================] - 3s 353ms/step - loss: 0.0048 - acc: 0.9973
Epoch 81/100
7/7 [==============================] - 2s 331ms/step - loss: 0.0046 - acc: 0.9971
Epoch 82/100
7/7 [==============================] - 2s 339ms/step - loss: 0.0047 - acc: 0.9971
Epoch 83/100
7/7 [==============================] - 2s 333ms/step - loss: 0.0045 - acc: 0.9973
Epoch 84/100
7/7 [==============================] - 2s 320ms/step - loss: 0.0047 - acc: 0.9973
Epoch 85/100
7/7 [==============================] - 2s 322ms/step - loss: 0.0045 - acc: 0.9973
Epoch 86/100
7/7 [==============================] - 2s 331ms/st

7/7 [==============================] - 2s 354ms/step - loss: 0.0037 - acc: 0.9981
Epoch 49/100
7/7 [==============================] - 3s 365ms/step - loss: 0.0039 - acc: 0.9981
Epoch 50/100
7/7 [==============================] - 2s 336ms/step - loss: 0.0039 - acc: 0.9981
Epoch 51/100
7/7 [==============================] - 2s 339ms/step - loss: 0.0037 - acc: 0.9981
Epoch 52/100
7/7 [==============================] - 2s 350ms/step - loss: 0.0037 - acc: 0.9981
Epoch 53/100
7/7 [==============================] - 2s 327ms/step - loss: 0.0036 - acc: 0.9981
Epoch 54/100
7/7 [==============================] - 2s 329ms/step - loss: 0.0037 - acc: 0.9979
Epoch 55/100
7/7 [==============================] - 2s 351ms/step - loss: 0.0042 - acc: 0.9981
Epoch 56/100
7/7 [==============================] - 2s 322ms/step - loss: 0.0036 - acc: 0.9981
Epoch 57/100
7/7 [==============================] - 2s 340ms/step - loss: 0.0037 - acc: 0.9981
Epoch 58/100
7/7 [==============================] - 2s 328ms/st

...

7/7 [==============================] - 1s 164ms/step - loss: 0.0035 - acc: 0.9979
Epoch 47/100
7/7 [==============================] - 1s 161ms/step - loss: 0.0035 - acc: 0.9979
Epoch 48/100
7/7 [==============================] - 1s 162ms/step - loss: 0.0035 - acc: 0.9979
Epoch 49/100
7/7 [==============================] - 1s 158ms/step - loss: 0.0035 - acc: 0.9979
Epoch 50/100
7/7 [==============================] - 1s 165ms/step - loss: 0.0035 - acc: 0.9979
Epoch 51/100
7/7 [==============================] - 1s 161ms/step - loss: 0.0035 - acc: 0.9979
Epoch 52/100
7/7 [==============================] - 1s 159ms/step - loss: 0.0035 - acc: 0.9979
Epoch 53/100
7/7 [==============================] - 1s 164ms/step - loss: 0.0036 - acc: 0.9979
Epoch 54/100
7/7 [==============================] - 1s 157ms/step - loss: 0.0035 - acc: 0.9979
Epoch 55/100
7/7 [==============================] - 1s 160ms/step - loss: 0.0035 - acc: 0.9979
Epoch 56/100
7/7 [==============================] - 1s 155ms/st

7/7 [==============================] - 2s 246ms/step - loss: 0.0034 - acc: 0.9981
Epoch 19/100
7/7 [==============================] - 1s 160ms/step - loss: 0.0034 - acc: 0.9981
Epoch 20/100
7/7 [==============================] - 2s 244ms/step - loss: 0.0033 - acc: 0.9981
Epoch 21/100
7/7 [==============================] - 1s 156ms/step - loss: 0.0033 - acc: 0.9981
Epoch 22/100
7/7 [==============================] - 2s 252ms/step - loss: 0.0034 - acc: 0.9981
Epoch 23/100
7/7 [==============================] - 1s 164ms/step - loss: 0.0034 - acc: 0.9981
Epoch 24/100
7/7 [==============================] - 2s 227ms/step - loss: 0.0033 - acc: 0.9981
Epoch 25/100
7/7 [==============================] - 1s 167ms/step - loss: 0.0034 - acc: 0.9981
Epoch 26/100
7/7 [==============================] - 2s 222ms/step - loss: 0.0033 - acc: 0.9981
Epoch 27/100
7/7 [==============================] - 1s 180ms/step - loss: 0.0032 - acc: 0.9981
Epoch 28/100
7/7 [==============================] - 2s 223ms/st

7/7 [==============================] - 2s 221ms/step - loss: 0.0033 - acc: 0.9981
Epoch 91/100
7/7 [==============================] - 1s 153ms/step - loss: 0.0033 - acc: 0.9981
Epoch 92/100
7/7 [==============================] - 2s 221ms/step - loss: 0.0033 - acc: 0.9981
Epoch 93/100
7/7 [==============================] - 1s 162ms/step - loss: 0.0033 - acc: 0.9981
Epoch 94/100
7/7 [==============================] - 1s 213ms/step - loss: 0.0033 - acc: 0.9981
Epoch 95/100
7/7 [==============================] - 1s 160ms/step - loss: 0.0034 - acc: 0.9981
Epoch 96/100
7/7 [==============================] - 1s 205ms/step - loss: 0.0033 - acc: 0.9981
Epoch 97/100
7/7 [==============================] - 1s 172ms/step - loss: 0.0033 - acc: 0.9981
Epoch 98/100
7/7 [==============================] - 1s 153ms/step - loss: 0.0033 - acc: 0.9981
Epoch 99/100
7/7 [==============================] - 2s 231ms/step - loss: 0.0034 - acc: 0.9981
Epoch 100/100
7/7 [==============================] - 1s 158ms/s

7/7 [==============================] - 1s 178ms/step - loss: 0.0038 - acc: 0.9976
Epoch 63/100
7/7 [==============================] - 2s 209ms/step - loss: 0.0039 - acc: 0.9976
Epoch 64/100
7/7 [==============================] - 1s 197ms/step - loss: 0.0038 - acc: 0.9976
Epoch 65/100
7/7 [==============================] - 1s 198ms/step - loss: 0.0038 - acc: 0.9976
Epoch 66/100
7/7 [==============================] - 1s 158ms/step - loss: 0.0040 - acc: 0.9976
Epoch 67/100
7/7 [==============================] - 1s 161ms/step - loss: 0.0038 - acc: 0.9976
Epoch 68/100
7/7 [==============================] - 1s 164ms/step - loss: 0.0037 - acc: 0.9976
Epoch 69/100
7/7 [==============================] - 1s 163ms/step - loss: 0.0036 - acc: 0.9978
Epoch 70/100
7/7 [==============================] - 1s 159ms/step - loss: 0.0036 - acc: 0.9978
Epoch 71/100
7/7 [==============================] - 1s 165ms/step - loss: 0.0037 - acc: 0.9978
Epoch 72/100
7/7 [==============================] - 1s 157ms/st

7/7 [==============================] - 1s 161ms/step - loss: 0.0033 - acc: 0.9981
Epoch 35/100
7/7 [==============================] - 1s 161ms/step - loss: 0.0033 - acc: 0.9981
Epoch 36/100
7/7 [==============================] - 1s 156ms/step - loss: 0.0033 - acc: 0.9981
Epoch 37/100
7/7 [==============================] - 1s 163ms/step - loss: 0.0033 - acc: 0.9981
Epoch 38/100
7/7 [==============================] - 1s 160ms/step - loss: 0.0033 - acc: 0.9981
Epoch 39/100
7/7 [==============================] - 1s 162ms/step - loss: 0.0033 - acc: 0.9981
Epoch 40/100
7/7 [==============================] - 1s 160ms/step - loss: 0.0033 - acc: 0.9981
Epoch 41/100
7/7 [==============================] - 1s 156ms/step - loss: 0.0033 - acc: 0.9981
Epoch 42/100
7/7 [==============================] - 1s 159ms/step - loss: 0.0034 - acc: 0.9981
Epoch 43/100
7/7 [==============================] - 1s 157ms/step - loss: 0.0033 - acc: 0.9981
Epoch 44/100
7/7 [==============================] - 1s 165ms/st

In [25]:
preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds),np.sqrt(np.mean(np.square(y_test-preds))))

정확도 : 0.7224358974358974 0.5268435275905955


In [12]:
import random
_list = list(range(1560))
random.shuffle(_list)
patient = {}
for i in range(1560):
    patient[i] = _list[i]

In [19]:
import time
from sklearn import metrics 
def item_acc(num):
    df = X_test.copy()
    for i in range(1560):
        df[i][:,num] = df[patient[i]][:,num]

    preds = final_model.predict(df)
    preds[preds>0.5]=1
    preds[preds<=0.5]=0
    acc = metrics.accuracy_score(y_test, preds)
    loss = np.sqrt(np.mean(np.square(y_test-preds)))
    print(f'item idx {num} acc & loss :', acc, loss)
    return [acc, loss]

In [20]:
item_acc(1)

item idx 1 acc & loss : 0.7224358974358974 0.5268435275905955


[0.7224358974358974, 0.5268435275905955]

In [31]:
import time
start = time.time()  # 시작 시간 : 7:00
result ={}
loss = {}
for i in range(4069):    
    temp = item_acc(i)
    result[i] = temp[0]
    loss[i] = temp[1]
    
print("time :", (time.time() - start)/60, '분') 

item idx 0 acc & loss : 0.7275641025641025 0.5219539227133918
item idx 1 acc & loss : 0.7224358974358974 0.5268435275905955
item idx 2 acc & loss : 0.725 0.5244044240850758
item idx 3 acc & loss : 0.7224358974358974 0.5268435275905955
item idx 4 acc & loss : 0.7205128205128205 0.5286654702996778
item idx 5 acc & loss : 0.7211538461538461 0.5280588545286916
item idx 6 acc & loss : 0.7230769230769231 0.5262348115842176
item idx 7 acc & loss : 0.7211538461538461 0.5280588545286916
item idx 8 acc & loss : 0.7185897435897436 0.5304811555656397
item idx 9 acc & loss : 0.7224358974358974 0.5268435275905955
item idx 10 acc & loss : 0.725 0.5244044240850758
item idx 11 acc & loss : 0.7185897435897436 0.5304811555656397
item idx 12 acc & loss : 0.7192307692307692 0.5298766184398315
item idx 13 acc & loss : 0.7217948717948718 0.5274515410965525
item idx 14 acc & loss : 0.7230769230769231 0.5262348115842176
item idx 15 acc & loss : 0.7224358974358974 0.5268435275905955
item idx 16 acc & loss : 0.7

KeyboardInterrupt: 

In [32]:
idx = [ 206,  121,   56,   20,  103,   34,  119,  184,  203,  204,   26,
               8,   48,   95, 3833,  187,   41, 2612,  186,  278,  197,  193,
               5,   94,   82,   19, 2196,    6, 2213,  305,   54, 2634,   86,
              21,   32, 3046,  188,  284,  861,  217,    2, 3457,  845,  124,
              11,  307,  619,    7, 2244,  191,   10, 1979,  182,   12, 1995,
            2190, 2050, 1993, 2149, 1994, 2453, 2488, 3722,   13, 2027, 3114,
            3104, 1828,  155, 2003,  162,   83, 2341, 1583, 2389, 1081,  621,
            3554, 2022, 1998,  383,   63, 2222, 3034, 1904,   81, 2327,   71,
            1981,    0, 2627, 3469,   25, 2291, 2107, 1000, 3219, 3899,  860,
            1034]

In [33]:
import time
start = time.time()  # 시작 시간 : 7:48
result ={}
loss = {}
for i in idx:    
    temp = item_acc(i)
    result[i] = temp[0]
    loss[i] = temp[1]
    
print("time :", (time.time() - start)/60, '분') 

item idx 206 acc & loss : 0.7147435897435898 0.5340940088190563
item idx 121 acc & loss : 0.7108974358974359 0.5376825867578047
item idx 56 acc & loss : 0.7064102564102565 0.5418392230078435
item idx 20 acc & loss : 0.7185897435897436 0.5304811555656397
item idx 103 acc & loss : 0.7198717948717949 0.52927139080835
item idx 34 acc & loss : 0.7211538461538461 0.5280588545286916
item idx 119 acc & loss : 0.7128205128205128 0.5358913016456669
item idx 184 acc & loss : 0.7282051282051282 0.5213394976355348
item idx 203 acc & loss : 0.717948717948718 0.5310850045437944
item idx 204 acc & loss : 0.7198717948717949 0.52927139080835
item idx 26 acc & loss : 0.7224358974358974 0.5268435275905955
item idx 8 acc & loss : 0.7185897435897436 0.5304811555656397
item idx 48 acc & loss : 0.7237179487179487 0.5256253906367645
item idx 95 acc & loss : 0.7230769230769231 0.5262348115842176
item idx 3833 acc & loss : 0.7256410256410256 0.5237928735282434
item idx 187 acc & loss : 0.7237179487179487 0.52562

In [34]:
result

{206: 0.7147435897435898,
 121: 0.7108974358974359,
 56: 0.7064102564102565,
 20: 0.7185897435897436,
 103: 0.7198717948717949,
 34: 0.7211538461538461,
 119: 0.7128205128205128,
 184: 0.7282051282051282,
 203: 0.717948717948718,
 204: 0.7198717948717949,
 26: 0.7224358974358974,
 8: 0.7185897435897436,
 48: 0.7237179487179487,
 95: 0.7230769230769231,
 3833: 0.7256410256410256,
 187: 0.7237179487179487,
 41: 0.7134615384615385,
 2612: 0.7185897435897436,
 186: 0.7166666666666667,
 278: 0.7205128205128205,
 197: 0.7269230769230769,
 193: 0.7243589743589743,
 5: 0.7211538461538461,
 94: 0.7121794871794872,
 82: 0.725,
 19: 0.7217948717948718,
 2196: 0.7173076923076923,
 6: 0.7230769230769231,
 2213: 0.7224358974358974,
 305: 0.725,
 54: 0.7243589743589743,
 2634: 0.7269230769230769,
 86: 0.7198717948717949,
 21: 0.7224358974358974,
 32: 0.7230769230769231,
 3046: 0.7198717948717949,
 188: 0.7243589743589743,
 284: 0.7211538461538461,
 861: 0.7217948717948718,
 217: 0.7224358974358974,
 

In [35]:
loss

{206: 0.5340940088190563,
 121: 0.5376825867578047,
 56: 0.5418392230078435,
 20: 0.5304811555656397,
 103: 0.52927139080835,
 34: 0.5280588545286916,
 119: 0.5358913016456669,
 184: 0.5213394976355348,
 203: 0.5310850045437944,
 204: 0.52927139080835,
 26: 0.5268435275905955,
 8: 0.5304811555656397,
 48: 0.5256253906367645,
 95: 0.5262348115842176,
 3833: 0.5237928735282434,
 187: 0.5256253906367645,
 41: 0.5352928745448248,
 2612: 0.5304811555656397,
 186: 0.5322906474223771,
 278: 0.5286654702996778,
 197: 0.5225676253624243,
 193: 0.5250152622934172,
 5: 0.5280588545286916,
 94: 0.5364890612309936,
 82: 0.5244044240850758,
 19: 0.5274515410965525,
 2196: 0.531688167718925,
 6: 0.5262348115842176,
 2213: 0.5268435275905955,
 305: 0.5244044240850758,
 54: 0.5250152622934172,
 2634: 0.5225676253624243,
 86: 0.52927139080835,
 21: 0.5268435275905955,
 32: 0.5262348115842176,
 3046: 0.52927139080835,
 188: 0.5250152622934172,
 284: 0.5280588545286916,
 861: 0.5274515410965525,
 217: 0.5

In [36]:
np.save('./result.npy', result)
np.save('./loss.npy', loss)

In [40]:
np.load('./result.npy',allow_pickle=True).item()
np.load('./loss.npy',allow_pickle=True).item()

{206: 0.5340940088190563,
 121: 0.5376825867578047,
 56: 0.5418392230078435,
 20: 0.5304811555656397,
 103: 0.52927139080835,
 34: 0.5280588545286916,
 119: 0.5358913016456669,
 184: 0.5213394976355348,
 203: 0.5310850045437944,
 204: 0.52927139080835,
 26: 0.5268435275905955,
 8: 0.5304811555656397,
 48: 0.5256253906367645,
 95: 0.5262348115842176,
 3833: 0.5237928735282434,
 187: 0.5256253906367645,
 41: 0.5352928745448248,
 2612: 0.5304811555656397,
 186: 0.5322906474223771,
 278: 0.5286654702996778,
 197: 0.5225676253624243,
 193: 0.5250152622934172,
 5: 0.5280588545286916,
 94: 0.5364890612309936,
 82: 0.5244044240850758,
 19: 0.5274515410965525,
 2196: 0.531688167718925,
 6: 0.5262348115842176,
 2213: 0.5268435275905955,
 305: 0.5244044240850758,
 54: 0.5250152622934172,
 2634: 0.5225676253624243,
 86: 0.52927139080835,
 21: 0.5268435275905955,
 32: 0.5262348115842176,
 3046: 0.52927139080835,
 188: 0.5250152622934172,
 284: 0.5280588545286916,
 861: 0.5274515410965525,
 217: 0.5

In [47]:
dict(sorted(loss.items(), key=lambda item: item[1]))

{1583: 0.5188745216627708,
 184: 0.5213394976355348,
 2389: 0.5219539227133918,
 0: 0.5219539227133918,
 197: 0.5225676253624243,
 2634: 0.5225676253624243,
 124: 0.5231806081249082,
 1828: 0.5231806081249082,
 3833: 0.5237928735282434,
 2050: 0.5237928735282434,
 2027: 0.5237928735282434,
 3104: 0.5237928735282434,
 155: 0.5237928735282434,
 82: 0.5244044240850758,
 305: 0.5244044240850758,
 2: 0.5244044240850758,
 10: 0.5244044240850758,
 193: 0.5250152622934172,
 54: 0.5250152622934172,
 188: 0.5250152622934172,
 1979: 0.5250152622934172,
 1994: 0.5250152622934172,
 162: 0.5250152622934172,
 2022: 0.5250152622934172,
 383: 0.5250152622934172,
 48: 0.5256253906367645,
 187: 0.5256253906367645,
 3457: 0.5256253906367645,
 1993: 0.5256253906367645,
 3722: 0.5256253906367645,
 83: 0.5256253906367645,
 2327: 0.5256253906367645,
 3469: 0.5256253906367645,
 95: 0.5262348115842176,
 6: 0.5262348115842176,
 32: 0.5262348115842176,
 307: 0.5262348115842176,
 619: 0.5262348115842176,
 2190: 0.

In [48]:
dict(sorted(result.items(), key=lambda item: item[1]))

{56: 0.7064102564102565,
 121: 0.7108974358974359,
 94: 0.7121794871794872,
 119: 0.7128205128205128,
 621: 0.7128205128205128,
 41: 0.7134615384615385,
 1995: 0.7141025641025641,
 206: 0.7147435897435898,
 186: 0.7166666666666667,
 2196: 0.7173076923076923,
 182: 0.7173076923076923,
 203: 0.717948717948718,
 1904: 0.717948717948718,
 20: 0.7185897435897436,
 8: 0.7185897435897436,
 2612: 0.7185897435897436,
 11: 0.7185897435897436,
 2341: 0.7185897435897436,
 71: 0.7185897435897436,
 12: 0.7192307692307692,
 1000: 0.7192307692307692,
 103: 0.7198717948717949,
 204: 0.7198717948717949,
 86: 0.7198717948717949,
 3046: 0.7198717948717949,
 191: 0.7198717948717949,
 278: 0.7205128205128205,
 2003: 0.7205128205128205,
 3554: 0.7205128205128205,
 34: 0.7211538461538461,
 5: 0.7211538461538461,
 284: 0.7211538461538461,
 7: 0.7211538461538461,
 2149: 0.7211538461538461,
 3114: 0.7211538461538461,
 2222: 0.7211538461538461,
 1981: 0.7211538461538461,
 3899: 0.7211538461538461,
 860: 0.7211538

In [ ]:
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(units=128, return_sequences=True))
model.add(LSTM(units=64, return_sequences=True))
model.add(LSTM(units=32, return_sequences=True))
model.add(LSTM(units=32, return_sequences=False))
# model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])
model.summary()

estimator = []
for i in range(1,51):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:model, epochs=50, batch_size=256)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))

In [8]:
from sklearn.metrics import roc_auc_score

print(' roc auc value : ', roc_auc_score(y_test, preds))

 roc auc value :  0.7383199079401611


In [11]:
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(units=128, activation='sigmoid', return_sequences=True))
model.add(LSTM(units=64, activation='sigmoid', return_sequences=True))
model.add(LSTM(units=32, activation='sigmoid', return_sequences=True))
model.add(LSTM(units=32, activation='sigmoid', return_sequences=False))
# model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])
model.summary()

estimator = []
for i in range(1,51):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:model, epochs=50, batch_size=1024)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_16 (LSTM)               (None, 10, 128)           2149376   
_________________________________________________________________
lstm_17 (LSTM)               (None, 10, 64)            49408     
_________________________________________________________________
lstm_18 (LSTM)               (None, 10, 32)            12416     
_________________________________________________________________
lstm_19 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 2,219,553
Trainable params: 2,219,553
Non-trainable params: 0
____________________________________________

7/7 [==============================] - 1s 176ms/step - loss: 0.0449 - acc: 0.9603
Epoch 18/50
7/7 [==============================] - 1s 175ms/step - loss: 0.0466 - acc: 0.9590
Epoch 19/50
7/7 [==============================] - 1s 163ms/step - loss: 0.0461 - acc: 0.9606
Epoch 20/50
7/7 [==============================] - 1s 161ms/step - loss: 0.0456 - acc: 0.9611
Epoch 21/50
7/7 [==============================] - 1s 174ms/step - loss: 0.0471 - acc: 0.9586
Epoch 22/50
7/7 [==============================] - 1s 173ms/step - loss: 0.0458 - acc: 0.9601
Epoch 23/50
7/7 [==============================] - 1s 169ms/step - loss: 0.0450 - acc: 0.9609
Epoch 24/50
7/7 [==============================] - 1s 170ms/step - loss: 0.0456 - acc: 0.9607
Epoch 25/50
7/7 [==============================] - 1s 165ms/step - loss: 0.0457 - acc: 0.9606
Epoch 26/50
7/7 [==============================] - 1s 174ms/step - loss: 0.0455 - acc: 0.9572
Epoch 27/50
7/7 [==============================] - 1s 177ms/step - loss:

7/7 [==============================] - 1s 158ms/step - loss: 0.0364 - acc: 0.9662
Epoch 42/50
7/7 [==============================] - 1s 159ms/step - loss: 0.0361 - acc: 0.9667
Epoch 43/50
7/7 [==============================] - 1s 159ms/step - loss: 0.0355 - acc: 0.9667
Epoch 44/50
7/7 [==============================] - 1s 158ms/step - loss: 0.0358 - acc: 0.9670
Epoch 45/50
7/7 [==============================] - 1s 160ms/step - loss: 0.0359 - acc: 0.9671
Epoch 46/50
7/7 [==============================] - 1s 160ms/step - loss: 0.0359 - acc: 0.9670
Epoch 47/50
7/7 [==============================] - 1s 159ms/step - loss: 0.0357 - acc: 0.9670
Epoch 48/50
7/7 [==============================] - 1s 158ms/step - loss: 0.0357 - acc: 0.9678
Epoch 49/50
7/7 [==============================] - 1s 158ms/step - loss: 0.0364 - acc: 0.9668
Epoch 50/50
7/7 [==============================] - 1s 159ms/step - loss: 0.0366 - acc: 0.9657
Epoch 1/50
7/7 [==============================] - 2s 167ms/step - loss: 

7/7 [==============================] - 1s 171ms/step - loss: 0.0332 - acc: 0.9691
Epoch 16/50
7/7 [==============================] - 1s 172ms/step - loss: 0.0327 - acc: 0.9695
Epoch 17/50
7/7 [==============================] - 1s 170ms/step - loss: 0.0343 - acc: 0.9679
Epoch 18/50
7/7 [==============================] - 1s 173ms/step - loss: 0.0344 - acc: 0.9683
Epoch 19/50
7/7 [==============================] - 1s 165ms/step - loss: 0.0330 - acc: 0.9692
Epoch 20/50
7/7 [==============================] - 1s 173ms/step - loss: 0.0333 - acc: 0.9691
Epoch 21/50
7/7 [==============================] - 1s 177ms/step - loss: 0.0341 - acc: 0.9686
Epoch 22/50
7/7 [==============================] - 1s 166ms/step - loss: 0.0326 - acc: 0.9692
Epoch 23/50
7/7 [==============================] - 1s 175ms/step - loss: 0.0334 - acc: 0.9678
Epoch 24/50
7/7 [==============================] - 1s 164ms/step - loss: 0.0340 - acc: 0.9683
Epoch 25/50
7/7 [==============================] - 1s 171ms/step - loss:

7/7 [==============================] - 1s 156ms/step - loss: 0.0254 - acc: 0.9703
Epoch 40/50
7/7 [==============================] - 1s 154ms/step - loss: 0.0251 - acc: 0.9695
Epoch 41/50
7/7 [==============================] - 1s 156ms/step - loss: 0.0241 - acc: 0.9702
Epoch 42/50
7/7 [==============================] - 1s 157ms/step - loss: 0.0242 - acc: 0.9699
Epoch 43/50
7/7 [==============================] - 1s 156ms/step - loss: 0.0241 - acc: 0.9702
Epoch 44/50
7/7 [==============================] - 1s 155ms/step - loss: 0.0234 - acc: 0.9703
Epoch 45/50
7/7 [==============================] - 1s 156ms/step - loss: 0.0240 - acc: 0.9699
Epoch 46/50
7/7 [==============================] - 1s 156ms/step - loss: 0.0240 - acc: 0.9691
Epoch 47/50
7/7 [==============================] - 1s 155ms/step - loss: 0.0236 - acc: 0.9702
Epoch 48/50
7/7 [==============================] - 1s 155ms/step - loss: 0.0228 - acc: 0.9702
Epoch 49/50
7/7 [==============================] - 1s 155ms/step - loss:

...

7/7 [==============================] - 1s 159ms/step - loss: 0.0079 - acc: 0.9910
Epoch 34/50
7/7 [==============================] - 1s 155ms/step - loss: 0.0076 - acc: 0.9921
Epoch 35/50
7/7 [==============================] - 1s 159ms/step - loss: 0.0073 - acc: 0.9931
Epoch 36/50
7/7 [==============================] - 1s 158ms/step - loss: 0.0072 - acc: 0.9944
Epoch 37/50
7/7 [==============================] - 1s 156ms/step - loss: 0.0073 - acc: 0.9926
Epoch 38/50
7/7 [==============================] - 1s 159ms/step - loss: 0.0076 - acc: 0.9923
Epoch 39/50
7/7 [==============================] - 1s 157ms/step - loss: 0.0069 - acc: 0.9944
Epoch 40/50
7/7 [==============================] - 1s 159ms/step - loss: 0.0075 - acc: 0.9934
Epoch 41/50
7/7 [==============================] - 1s 159ms/step - loss: 0.0077 - acc: 0.9917
Epoch 42/50
7/7 [==============================] - 1s 157ms/step - loss: 0.0072 - acc: 0.9925
Epoch 43/50
7/7 [==============================] - 1s 158ms/step - loss:

7/7 [==============================] - 1s 169ms/step - loss: 0.0073 - acc: 0.9970
Epoch 8/50
7/7 [==============================] - 1s 167ms/step - loss: 0.0071 - acc: 0.9965
Epoch 9/50
7/7 [==============================] - 1s 171ms/step - loss: 0.0072 - acc: 0.9968
Epoch 10/50
7/7 [==============================] - 1s 172ms/step - loss: 0.0068 - acc: 0.9966
Epoch 11/50
7/7 [==============================] - 1s 170ms/step - loss: 0.0071 - acc: 0.9960
Epoch 12/50
7/7 [==============================] - 1s 163ms/step - loss: 0.0071 - acc: 0.9966
Epoch 13/50
7/7 [==============================] - 1s 168ms/step - loss: 0.0071 - acc: 0.9965
Epoch 14/50
7/7 [==============================] - 1s 177ms/step - loss: 0.0071 - acc: 0.9963
Epoch 15/50
7/7 [==============================] - 1s 171ms/step - loss: 0.0071 - acc: 0.9965
Epoch 16/50
7/7 [==============================] - 1s 174ms/step - loss: 0.0071 - acc: 0.9966
Epoch 17/50
7/7 [==============================] - 1s 165ms/step - loss: 0

7/7 [==============================] - 1s 165ms/step - loss: 0.0070 - acc: 0.9965
Epoch 32/50
7/7 [==============================] - 1s 175ms/step - loss: 0.0068 - acc: 0.9965
Epoch 33/50
7/7 [==============================] - 1s 155ms/step - loss: 0.0068 - acc: 0.9962
Epoch 34/50
7/7 [==============================] - 1s 155ms/step - loss: 0.0068 - acc: 0.9965
Epoch 35/50
7/7 [==============================] - 1s 157ms/step - loss: 0.0067 - acc: 0.9970
Epoch 36/50
7/7 [==============================] - 1s 156ms/step - loss: 0.0066 - acc: 0.9973
Epoch 37/50
7/7 [==============================] - 1s 156ms/step - loss: 0.0070 - acc: 0.9968
Epoch 38/50
7/7 [==============================] - 1s 158ms/step - loss: 0.0068 - acc: 0.9968
Epoch 39/50
7/7 [==============================] - 1s 162ms/step - loss: 0.0068 - acc: 0.9968
Epoch 40/50
7/7 [==============================] - 1s 155ms/step - loss: 0.0064 - acc: 0.9968
Epoch 41/50
7/7 [==============================] - 1s 158ms/step - loss:

7/7 [==============================] - 1s 161ms/step - loss: 0.0067 - acc: 0.9974
Epoch 6/50
7/7 [==============================] - 1s 169ms/step - loss: 0.0066 - acc: 0.9973
Epoch 7/50
7/7 [==============================] - 1s 163ms/step - loss: 0.0069 - acc: 0.9971
Epoch 8/50
7/7 [==============================] - 1s 162ms/step - loss: 0.0067 - acc: 0.9973
Epoch 9/50
7/7 [==============================] - 1s 169ms/step - loss: 0.0069 - acc: 0.9971
Epoch 10/50
7/7 [==============================] - 1s 164ms/step - loss: 0.0070 - acc: 0.9970
Epoch 11/50
7/7 [==============================] - 1s 163ms/step - loss: 0.0076 - acc: 0.9960
Epoch 12/50
7/7 [==============================] - 1s 165ms/step - loss: 0.0068 - acc: 0.9973
Epoch 13/50
7/7 [==============================] - 1s 168ms/step - loss: 0.0069 - acc: 0.9970
Epoch 14/50
7/7 [==============================] - 1s 170ms/step - loss: 0.0068 - acc: 0.9973
Epoch 15/50
7/7 [==============================] - 1s 160ms/step - loss: 0.0

7/7 [==============================] - 1s 162ms/step - loss: 0.0066 - acc: 0.9973
Epoch 30/50
7/7 [==============================] - 1s 159ms/step - loss: 0.0064 - acc: 0.9974
Epoch 31/50
7/7 [==============================] - 1s 163ms/step - loss: 0.0065 - acc: 0.9974
Epoch 32/50
7/7 [==============================] - 1s 157ms/step - loss: 0.0066 - acc: 0.9974
Epoch 33/50
7/7 [==============================] - 1s 153ms/step - loss: 0.0067 - acc: 0.9973
Epoch 34/50
7/7 [==============================] - 1s 152ms/step - loss: 0.0070 - acc: 0.9970
Epoch 35/50
7/7 [==============================] - 1s 155ms/step - loss: 0.0097 - acc: 0.9937
Epoch 36/50
7/7 [==============================] - 1s 154ms/step - loss: 0.0085 - acc: 0.9949
Epoch 37/50
7/7 [==============================] - 1s 156ms/step - loss: 0.0071 - acc: 0.9963
Epoch 38/50
7/7 [==============================] - 1s 156ms/step - loss: 0.0081 - acc: 0.9947
Epoch 39/50
7/7 [==============================] - 1s 157ms/step - loss:

In [14]:
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(units=128, activation='sigmoid', recurrent_activation='hard_sigmoid', return_sequences=True))
model.add(LSTM(units=64, activation='sigmoid', recurrent_activation='hard_sigmoid', return_sequences=True))
model.add(LSTM(units=32, activation='sigmoid', recurrent_activation='hard_sigmoid', return_sequences=True))
model.add(LSTM(units=32, activation='sigmoid', recurrent_activation='hard_sigmoid', return_sequences=False))
# model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])
model.summary()

estimator = []
for i in range(1,51):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:model, epochs=50, batch_size=1024)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_24 (LSTM)               (None, 10, 128)           2149376   
_________________________________________________________________
lstm_25 (LSTM)               (None, 10, 64)            49408     
_________________________________________________________________
lstm_26 (LSTM)               (None, 10, 32)            12416     
_________________________________________________________________
lstm_27 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 2,219,553
Trainable params: 2,219,553
Non-trainable params: 0
____________________________________________

7/7 [==============================] - 1s 184ms/step - loss: 0.0463 - acc: 0.9614
Epoch 18/50
7/7 [==============================] - 1s 186ms/step - loss: 0.0455 - acc: 0.9590
Epoch 19/50
7/7 [==============================] - 1s 183ms/step - loss: 0.0457 - acc: 0.9620
Epoch 20/50
7/7 [==============================] - 1s 188ms/step - loss: 0.0450 - acc: 0.9630
Epoch 21/50
7/7 [==============================] - 1s 196ms/step - loss: 0.0453 - acc: 0.9622
Epoch 22/50
7/7 [==============================] - 1s 190ms/step - loss: 0.0446 - acc: 0.9627
Epoch 23/50
7/7 [==============================] - 1s 185ms/step - loss: 0.0446 - acc: 0.9614
Epoch 24/50
7/7 [==============================] - 1s 184ms/step - loss: 0.0438 - acc: 0.9639
Epoch 25/50
7/7 [==============================] - 1s 185ms/step - loss: 0.0439 - acc: 0.9638
Epoch 26/50
7/7 [==============================] - 1s 194ms/step - loss: 0.0443 - acc: 0.9620
Epoch 27/50
7/7 [==============================] - 1s 185ms/step - loss:

7/7 [==============================] - 1s 180ms/step - loss: 0.0297 - acc: 0.9684
Epoch 42/50
7/7 [==============================] - 1s 182ms/step - loss: 0.0289 - acc: 0.9697
Epoch 43/50
7/7 [==============================] - 1s 181ms/step - loss: 0.0286 - acc: 0.9699
Epoch 44/50
7/7 [==============================] - 1s 181ms/step - loss: 0.0279 - acc: 0.9700
Epoch 45/50
7/7 [==============================] - 1s 178ms/step - loss: 0.0281 - acc: 0.9695
Epoch 46/50
7/7 [==============================] - 1s 178ms/step - loss: 0.0277 - acc: 0.9695
Epoch 47/50
7/7 [==============================] - 1s 179ms/step - loss: 0.0275 - acc: 0.9703
Epoch 48/50
7/7 [==============================] - 1s 179ms/step - loss: 0.0275 - acc: 0.9695
Epoch 49/50
7/7 [==============================] - 1s 180ms/step - loss: 0.0276 - acc: 0.9708
Epoch 50/50
7/7 [==============================] - 1s 180ms/step - loss: 0.0277 - acc: 0.9705
Epoch 1/50
7/7 [==============================] - 2s 186ms/step - loss: 

7/7 [==============================] - 1s 192ms/step - loss: 0.0185 - acc: 0.9820
Epoch 16/50
7/7 [==============================] - 1s 183ms/step - loss: 0.0195 - acc: 0.9811
Epoch 17/50
7/7 [==============================] - 1s 191ms/step - loss: 0.0192 - acc: 0.9816
Epoch 18/50
7/7 [==============================] - 1s 186ms/step - loss: 0.0199 - acc: 0.9803
Epoch 19/50
7/7 [==============================] - 1s 191ms/step - loss: 0.0194 - acc: 0.9809
Epoch 20/50
7/7 [==============================] - 1s 191ms/step - loss: 0.0189 - acc: 0.9814
Epoch 21/50
7/7 [==============================] - 1s 191ms/step - loss: 0.0196 - acc: 0.9811
Epoch 22/50
7/7 [==============================] - 1s 200ms/step - loss: 0.0182 - acc: 0.9819
Epoch 23/50
7/7 [==============================] - 1s 191ms/step - loss: 0.0183 - acc: 0.9816
Epoch 24/50
7/7 [==============================] - 1s 185ms/step - loss: 0.0189 - acc: 0.9809
Epoch 25/50
7/7 [==============================] - 1s 195ms/step - loss:

7/7 [==============================] - 1s 180ms/step - loss: 0.0115 - acc: 0.9950
Epoch 40/50
7/7 [==============================] - 1s 182ms/step - loss: 0.0117 - acc: 0.9949
Epoch 41/50
7/7 [==============================] - 1s 180ms/step - loss: 0.0116 - acc: 0.9952
Epoch 42/50
7/7 [==============================] - 1s 179ms/step - loss: 0.0119 - acc: 0.9949
Epoch 43/50
7/7 [==============================] - 1s 179ms/step - loss: 0.0113 - acc: 0.9947
Epoch 44/50
7/7 [==============================] - 1s 180ms/step - loss: 0.0115 - acc: 0.9954
Epoch 45/50
7/7 [==============================] - 1s 179ms/step - loss: 0.0116 - acc: 0.9947
Epoch 46/50
7/7 [==============================] - 1s 180ms/step - loss: 0.0113 - acc: 0.9950
Epoch 47/50
7/7 [==============================] - 1s 178ms/step - loss: 0.0116 - acc: 0.9952
Epoch 48/50
7/7 [==============================] - 1s 179ms/step - loss: 0.0111 - acc: 0.9950
Epoch 49/50
7/7 [==============================] - 1s 179ms/step - loss:

...

7/7 [==============================] - 1s 180ms/step - loss: 0.0091 - acc: 0.9978
Epoch 34/50
7/7 [==============================] - 1s 180ms/step - loss: 0.0091 - acc: 0.9978
Epoch 35/50
7/7 [==============================] - 1s 183ms/step - loss: 0.0091 - acc: 0.9978
Epoch 36/50
7/7 [==============================] - 1s 178ms/step - loss: 0.0092 - acc: 0.9978
Epoch 37/50
7/7 [==============================] - 1s 181ms/step - loss: 0.0095 - acc: 0.9978
Epoch 38/50
7/7 [==============================] - 1s 178ms/step - loss: 0.0107 - acc: 0.9957
Epoch 39/50
7/7 [==============================] - 1s 179ms/step - loss: 0.0146 - acc: 0.9915
Epoch 40/50
7/7 [==============================] - 1s 181ms/step - loss: 0.0190 - acc: 0.9857
Epoch 41/50
7/7 [==============================] - 1s 183ms/step - loss: 0.0155 - acc: 0.9891
Epoch 42/50
7/7 [==============================] - 1s 181ms/step - loss: 0.0152 - acc: 0.9901
Epoch 43/50
7/7 [==============================] - 1s 180ms/step - loss:

7/7 [==============================] - 1s 188ms/step - loss: 0.0090 - acc: 0.9978
Epoch 8/50
7/7 [==============================] - 1s 197ms/step - loss: 0.0091 - acc: 0.9978
Epoch 9/50
7/7 [==============================] - 1s 189ms/step - loss: 0.0093 - acc: 0.9978
Epoch 10/50
7/7 [==============================] - 1s 190ms/step - loss: 0.0091 - acc: 0.9978
Epoch 11/50
7/7 [==============================] - 1s 190ms/step - loss: 0.0092 - acc: 0.9978
Epoch 12/50
7/7 [==============================] - 1s 185ms/step - loss: 0.0091 - acc: 0.9978
Epoch 13/50
7/7 [==============================] - 1s 186ms/step - loss: 0.0092 - acc: 0.9978
Epoch 14/50
7/7 [==============================] - 1s 185ms/step - loss: 0.0094 - acc: 0.9978
Epoch 15/50
7/7 [==============================] - 1s 189ms/step - loss: 0.0090 - acc: 0.9978
Epoch 16/50
7/7 [==============================] - 1s 188ms/step - loss: 0.0092 - acc: 0.9978
Epoch 17/50
7/7 [==============================] - 1s 182ms/step - loss: 0

7/7 [==============================] - 1s 184ms/step - loss: 0.0091 - acc: 0.9978
Epoch 32/50
7/7 [==============================] - 1s 188ms/step - loss: 0.0090 - acc: 0.9978
Epoch 33/50
7/7 [==============================] - 1s 178ms/step - loss: 0.0091 - acc: 0.9978
Epoch 34/50
7/7 [==============================] - 1s 178ms/step - loss: 0.0089 - acc: 0.9978
Epoch 35/50
7/7 [==============================] - 1s 177ms/step - loss: 0.0089 - acc: 0.9979
Epoch 36/50
7/7 [==============================] - 1s 178ms/step - loss: 0.0091 - acc: 0.9978
Epoch 37/50
7/7 [==============================] - 1s 177ms/step - loss: 0.0087 - acc: 0.9979
Epoch 38/50
7/7 [==============================] - 1s 180ms/step - loss: 0.0089 - acc: 0.9979
Epoch 39/50
7/7 [==============================] - 1s 179ms/step - loss: 0.0090 - acc: 0.9979
Epoch 40/50
7/7 [==============================] - 1s 179ms/step - loss: 0.0089 - acc: 0.9979
Epoch 41/50
7/7 [==============================] - 1s 178ms/step - loss:

7/7 [==============================] - 1s 185ms/step - loss: 0.0090 - acc: 0.9979
Epoch 6/50
7/7 [==============================] - 1s 189ms/step - loss: 0.0090 - acc: 0.9979
Epoch 7/50
7/7 [==============================] - 1s 185ms/step - loss: 0.0089 - acc: 0.9979
Epoch 8/50
7/7 [==============================] - 1s 187ms/step - loss: 0.0088 - acc: 0.9979
Epoch 9/50
7/7 [==============================] - 1s 182ms/step - loss: 0.0087 - acc: 0.9979
Epoch 10/50
7/7 [==============================] - 1s 194ms/step - loss: 0.0089 - acc: 0.9979
Epoch 11/50
7/7 [==============================] - 1s 186ms/step - loss: 0.0088 - acc: 0.9979
Epoch 12/50
7/7 [==============================] - 1s 182ms/step - loss: 0.0090 - acc: 0.9979
Epoch 13/50
7/7 [==============================] - 1s 192ms/step - loss: 0.0090 - acc: 0.9979
Epoch 14/50
7/7 [==============================] - 1s 185ms/step - loss: 0.0091 - acc: 0.9979
Epoch 15/50
7/7 [==============================] - 1s 187ms/step - loss: 0.0

7/7 [==============================] - 1s 191ms/step - loss: 0.0089 - acc: 0.9979
Epoch 30/50
7/7 [==============================] - 1s 194ms/step - loss: 0.0090 - acc: 0.9979
Epoch 31/50
7/7 [==============================] - 1s 192ms/step - loss: 0.0090 - acc: 0.9979
Epoch 32/50
7/7 [==============================] - 1s 181ms/step - loss: 0.0090 - acc: 0.9979
Epoch 33/50
7/7 [==============================] - 1s 177ms/step - loss: 0.0089 - acc: 0.9979
Epoch 34/50
7/7 [==============================] - 1s 180ms/step - loss: 0.0089 - acc: 0.9979
Epoch 35/50
7/7 [==============================] - 1s 180ms/step - loss: 0.0087 - acc: 0.9979
Epoch 36/50
7/7 [==============================] - 1s 181ms/step - loss: 0.0088 - acc: 0.9979
Epoch 37/50
7/7 [==============================] - 1s 180ms/step - loss: 0.0086 - acc: 0.9979
Epoch 38/50
7/7 [==============================] - 1s 181ms/step - loss: 0.0091 - acc: 0.9979
Epoch 39/50
7/7 [==============================] - 1s 180ms/step - loss:

In [9]:
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(units=128, activation='relu', return_sequences=True))
model.add(LSTM(units=64, activation='relu', return_sequences=True))
model.add(LSTM(units=32, activation='relu', return_sequences=True))
model.add(LSTM(units=32, activation='relu', return_sequences=False))
# model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])
model.summary()

estimator = []
for i in range(1,51):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:model, epochs=100, batch_size=1024)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 10, 128)           2149376   
_________________________________________________________________
lstm_9 (LSTM)                (None, 10, 64)            49408     
_________________________________________________________________
lstm_10 (LSTM)               (None, 10, 32)            12416     
_________________________________________________________________
lstm_11 (LSTM)               (None, 32)                8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 33        
Total params: 2,219,553
Trainable params: 2,219,553
Non-trainable params: 0
____________________________________________

7/7 [==============================] - 2s 257ms/step - loss: 0.0127 - acc: 0.9950
Epoch 17/100
7/7 [==============================] - 2s 254ms/step - loss: 0.0126 - acc: 0.9957
Epoch 18/100
7/7 [==============================] - 2s 261ms/step - loss: 0.0123 - acc: 0.9960
Epoch 19/100
7/7 [==============================] - 2s 273ms/step - loss: 0.0120 - acc: 0.9954
Epoch 20/100
7/7 [==============================] - 2s 261ms/step - loss: 0.0120 - acc: 0.9958
Epoch 21/100
7/7 [==============================] - 2s 254ms/step - loss: 0.0120 - acc: 0.9958
Epoch 22/100
7/7 [==============================] - 2s 251ms/step - loss: 0.0116 - acc: 0.9962
Epoch 23/100
7/7 [==============================] - 2s 258ms/step - loss: 0.0115 - acc: 0.9962
Epoch 24/100
7/7 [==============================] - 2s 258ms/step - loss: 0.0118 - acc: 0.9962
Epoch 25/100
7/7 [==============================] - 2s 273ms/step - loss: 0.0116 - acc: 0.9962
Epoch 26/100
7/7 [==============================] - 2s 251ms/st

7/7 [==============================] - 2s 259ms/step - loss: 0.0078 - acc: 0.9966
Epoch 89/100
7/7 [==============================] - 2s 247ms/step - loss: 0.0080 - acc: 0.9966
Epoch 90/100
7/7 [==============================] - 2s 244ms/step - loss: 0.0080 - acc: 0.9968
Epoch 91/100
7/7 [==============================] - 2s 256ms/step - loss: 0.0079 - acc: 0.9966
Epoch 92/100
7/7 [==============================] - 2s 243ms/step - loss: 0.0078 - acc: 0.9966
Epoch 93/100
7/7 [==============================] - 2s 242ms/step - loss: 0.0079 - acc: 0.9968
Epoch 94/100
7/7 [==============================] - 2s 252ms/step - loss: 0.0079 - acc: 0.9966
Epoch 95/100
7/7 [==============================] - 2s 253ms/step - loss: 0.0078 - acc: 0.9966
Epoch 96/100
7/7 [==============================] - 2s 245ms/step - loss: 0.0077 - acc: 0.9966
Epoch 97/100
7/7 [==============================] - 2s 238ms/step - loss: 0.0079 - acc: 0.9966
Epoch 98/100
7/7 [==============================] - 2s 242ms/st

7/7 [==============================] - 2s 258ms/step - loss: 0.0092 - acc: 0.9946
Epoch 61/100
7/7 [==============================] - 2s 263ms/step - loss: 0.0080 - acc: 0.9958
Epoch 62/100
7/7 [==============================] - 2s 263ms/step - loss: 0.0076 - acc: 0.9968
Epoch 63/100
7/7 [==============================] - 2s 259ms/step - loss: 0.0069 - acc: 0.9974
Epoch 64/100
7/7 [==============================] - 2s 254ms/step - loss: 0.0072 - acc: 0.9968
Epoch 65/100
7/7 [==============================] - 2s 258ms/step - loss: 0.0068 - acc: 0.9974
Epoch 66/100
7/7 [==============================] - 2s 251ms/step - loss: 0.0068 - acc: 0.9974
Epoch 67/100
7/7 [==============================] - 2s 255ms/step - loss: 0.0065 - acc: 0.9976
Epoch 68/100
7/7 [==============================] - 2s 254ms/step - loss: 0.0064 - acc: 0.9976
Epoch 69/100
7/7 [==============================] - 2s 244ms/step - loss: 0.0066 - acc: 0.9976
Epoch 70/100
7/7 [==============================] - 2s 254ms/st

7/7 [==============================] - 2s 262ms/step - loss: 0.0062 - acc: 0.9974
Epoch 33/100
7/7 [==============================] - 2s 260ms/step - loss: 0.0060 - acc: 0.9978
Epoch 34/100
7/7 [==============================] - 2s 267ms/step - loss: 0.0061 - acc: 0.9979
Epoch 35/100
7/7 [==============================] - 2s 253ms/step - loss: 0.0060 - acc: 0.9976
Epoch 36/100
7/7 [==============================] - 2s 248ms/step - loss: 0.0062 - acc: 0.9978
Epoch 37/100
7/7 [==============================] - 2s 245ms/step - loss: 0.0060 - acc: 0.9978
Epoch 38/100
7/7 [==============================] - 2s 247ms/step - loss: 0.0060 - acc: 0.9978
Epoch 39/100
7/7 [==============================] - 2s 242ms/step - loss: 0.0061 - acc: 0.9978
Epoch 40/100
7/7 [==============================] - 2s 254ms/step - loss: 0.0060 - acc: 0.9978
Epoch 41/100
7/7 [==============================] - 2s 266ms/step - loss: 0.0060 - acc: 0.9979
Epoch 42/100
7/7 [==============================] - 2s 244ms/st

...

7/7 [==============================] - 2s 248ms/step - loss: 0.0069 - acc: 0.9962
Epoch 45/100
7/7 [==============================] - 2s 246ms/step - loss: 0.0067 - acc: 0.9960
Epoch 46/100
7/7 [==============================] - 2s 246ms/step - loss: 0.0066 - acc: 0.9963
Epoch 47/100
7/7 [==============================] - 2s 252ms/step - loss: 0.0064 - acc: 0.9963
Epoch 48/100
7/7 [==============================] - 2s 254ms/step - loss: 0.0064 - acc: 0.9963
Epoch 49/100
7/7 [==============================] - 2s 255ms/step - loss: 0.0065 - acc: 0.9962
Epoch 50/100
7/7 [==============================] - 2s 246ms/step - loss: 0.0066 - acc: 0.9963
Epoch 51/100
7/7 [==============================] - 2s 255ms/step - loss: 0.0066 - acc: 0.9963
Epoch 52/100
7/7 [==============================] - 2s 259ms/step - loss: 0.0065 - acc: 0.9963
Epoch 53/100
7/7 [==============================] - 2s 258ms/step - loss: 0.0067 - acc: 0.9963
Epoch 54/100
7/7 [==============================] - 2s 250ms/st

7/7 [==============================] - 2s 259ms/step - loss: 0.0057 - acc: 0.9974
Epoch 17/100
7/7 [==============================] - 2s 246ms/step - loss: 0.0057 - acc: 0.9974
Epoch 18/100
7/7 [==============================] - 2s 255ms/step - loss: 0.0055 - acc: 0.9974
Epoch 19/100
7/7 [==============================] - 2s 273ms/step - loss: 0.0058 - acc: 0.9974
Epoch 20/100
7/7 [==============================] - 2s 258ms/step - loss: 0.0055 - acc: 0.9974
Epoch 21/100
7/7 [==============================] - 2s 258ms/step - loss: 0.0056 - acc: 0.9974
Epoch 22/100
7/7 [==============================] - 2s 264ms/step - loss: 0.0056 - acc: 0.9973
Epoch 23/100
7/7 [==============================] - 2s 259ms/step - loss: 0.0056 - acc: 0.9974
Epoch 24/100
7/7 [==============================] - 2s 245ms/step - loss: 0.0056 - acc: 0.9974
Epoch 25/100
7/7 [==============================] - 2s 242ms/step - loss: 0.0056 - acc: 0.9973
Epoch 26/100
7/7 [==============================] - 2s 262ms/st

7/7 [==============================] - 2s 245ms/step - loss: 0.0051 - acc: 0.9981
Epoch 89/100
7/7 [==============================] - 2s 243ms/step - loss: 0.0051 - acc: 0.9981
Epoch 90/100
7/7 [==============================] - 2s 245ms/step - loss: 0.0050 - acc: 0.9981
Epoch 91/100
7/7 [==============================] - 2s 243ms/step - loss: 0.0051 - acc: 0.9981
Epoch 92/100
7/7 [==============================] - 2s 254ms/step - loss: 0.0051 - acc: 0.9981
Epoch 93/100
7/7 [==============================] - 2s 246ms/step - loss: 0.0050 - acc: 0.9981
Epoch 94/100
7/7 [==============================] - 2s 234ms/step - loss: 0.0051 - acc: 0.9981
Epoch 95/100
7/7 [==============================] - 2s 256ms/step - loss: 0.0051 - acc: 0.9981
Epoch 96/100
7/7 [==============================] - 2s 250ms/step - loss: 0.0051 - acc: 0.9981
Epoch 97/100
7/7 [==============================] - 2s 245ms/step - loss: 0.0052 - acc: 0.9981
Epoch 98/100
7/7 [==============================] - 2s 244ms/st

7/7 [==============================] - 2s 258ms/step - loss: 0.0052 - acc: 0.9981
Epoch 61/100
7/7 [==============================] - 2s 262ms/step - loss: 0.0052 - acc: 0.9981
Epoch 62/100
7/7 [==============================] - 2s 256ms/step - loss: 0.0052 - acc: 0.9981
Epoch 63/100
7/7 [==============================] - 2s 263ms/step - loss: 0.0050 - acc: 0.9981
Epoch 64/100
7/7 [==============================] - 2s 258ms/step - loss: 0.0050 - acc: 0.9981
Epoch 65/100
7/7 [==============================] - 2s 246ms/step - loss: 0.0053 - acc: 0.9979
Epoch 66/100
7/7 [==============================] - 2s 261ms/step - loss: 0.0052 - acc: 0.9981
Epoch 67/100
7/7 [==============================] - 2s 261ms/step - loss: 0.0050 - acc: 0.9981
Epoch 68/100
7/7 [==============================] - 2s 245ms/step - loss: 0.0050 - acc: 0.9981
Epoch 69/100
7/7 [==============================] - 2s 254ms/step - loss: 0.0050 - acc: 0.9981
Epoch 70/100
7/7 [==============================] - 2s 260ms/st

7/7 [==============================] - 2s 262ms/step - loss: 0.0050 - acc: 0.9981
Epoch 33/100
7/7 [==============================] - 2s 266ms/step - loss: 0.0050 - acc: 0.9981
Epoch 34/100
7/7 [==============================] - 2s 269ms/step - loss: 0.0051 - acc: 0.9981
Epoch 35/100
7/7 [==============================] - 2s 253ms/step - loss: 0.0050 - acc: 0.9981
Epoch 36/100
7/7 [==============================] - 2s 208ms/step - loss: 0.0051 - acc: 0.9981
Epoch 37/100
7/7 [==============================] - 2s 363ms/step - loss: 0.0050 - acc: 0.9981
Epoch 38/100
7/7 [==============================] - 2s 274ms/step - loss: 0.0052 - acc: 0.9981
Epoch 39/100
7/7 [==============================] - 2s 263ms/step - loss: 0.0051 - acc: 0.9981
Epoch 40/100
7/7 [==============================] - 2s 263ms/step - loss: 0.0052 - acc: 0.9981
Epoch 41/100
7/7 [==============================] - 2s 265ms/step - loss: 0.0053 - acc: 0.9981
Epoch 42/100
7/7 [==============================] - 2s 264ms/st

In [36]:
import time
start = time.time()  # 시작 시간 저장

model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(units=128, activation='relu', return_sequences=True))
model.add(LSTM(units=64, activation='relu', return_sequences=True))
model.add(LSTM(units=32, activation='relu', return_sequences=False))
# model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])
model.summary()

estimator = []
for i in range(1,11):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:model, epochs=20, batch_size=256)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print("time :", (time.time() - start)/60, '분') 

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_33 (LSTM)               (None, 10, 128)           2149376   
_________________________________________________________________
lstm_34 (LSTM)               (None, 10, 64)            49408     
_________________________________________________________________
lstm_35 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dropout_12 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 33        
Total params: 2,211,233
Trainable params: 2,211,233
Non-trainable params: 0
_________________________________________________________________
[('model1', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0d3efbca0>), ('model2', <keras.wrappe

Epoch 8/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0092 - acc: 0.9970
Epoch 9/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0086 - acc: 0.9973
Epoch 10/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0086 - acc: 0.9973
Epoch 11/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0087 - acc: 0.9973
Epoch 12/20
25/25 [==============================] - 2s 72ms/step - loss: 0.0087 - acc: 0.9971
Epoch 13/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0088 - acc: 0.9973
Epoch 14/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0091 - acc: 0.9970
Epoch 15/20
25/25 [==============================] - 2s 73ms/step - loss: 0.0091 - acc: 0.9971
Epoch 16/20
25/25 [==============================] - 2s 70ms/step - loss: 0.0086 - acc: 0.9973
Epoch 17/20
25/25 [==============================] - 2s 69ms/step - loss: 0.0087 - acc: 0.9971
Epoch 18/20
25/25 [==============================] -

In [16]:
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
# model.add(LSTM(units=128, return_sequences=True))
model.add(LSTM(units=64, return_sequences=False))
# model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 64)                1058304   
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 65        
Total params: 1,058,369
Trainable params: 1,058,369
Non-trainable params: 0
_________________________________________________________________


In [17]:
estimator = []
for i in range(1,3):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:model, epochs=20, batch_size=256)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

[('model1', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c27e9a0>), ('model2', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c27e850>)]
Epoch 1/20
25/25 [==============================] - 2s 29ms/step - loss: 0.2866 - acc: 0.6001
Epoch 2/20
25/25 [==============================] - 1s 29ms/step - loss: 0.1592 - acc: 0.7731
Epoch 3/20
25/25 [==============================] - 1s 29ms/step - loss: 0.1336 - acc: 0.8229
Epoch 4/20
25/25 [==============================] - 1s 28ms/step - loss: 0.1142 - acc: 0.8536
Epoch 5/20
25/25 [==============================] - 1s 29ms/step - loss: 0.0971 - acc: 0.8843
Epoch 6/20
25/25 [==============================] - 1s 29ms/step - loss: 0.0859 - acc: 0.9034
Epoch 7/20
25/25 [==============================] - 1s 30ms/step - loss: 0.0780 - acc: 0.9154
Epoch 8/20
25/25 [==============================] - 1s 31ms/step - loss: 0.0666 - acc: 0.9304
Epoch 9/20
25/25 [==============================] - 1s 31ms/step - loss: 0.0598

VotingClassifier(estimators=[('model1',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c27e9a0>),
                             ('model2',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c27e850>)],
                 voting='soft')

In [18]:
preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print(f"MAE : {MAE(y_test, preds)}, MSE : {MSE(y_test, preds)}, RMSE : {RMSE(y_test, preds)}")

정확도 : 0.7288461538461538
MAE : 2.71e-01, MSE : 2.71e-01, RMSE : 5.21e-01


---

In [18]:
# from keras.layers import *
# model = Sequential()
# model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
# model.add(LSTM(units=128, activation='relu', recurrent_activation='hard_sigmoid', return_sequences=True))
# model.add(BatchNormalization())
# model.add(LSTM(units=64, activation='relu', recurrent_activation='hard_sigmoid', return_sequences=True))
# model.add(BatchNormalization())
# model.add(LSTM(units=32, activation='relu', recurrent_activation='hard_sigmoid', return_sequences=False))
# # model.add(LSTM(units=32))
# model.add(Dropout(0.2))
# model.add(Dense(units=1))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['acc'])
# model.summary()

In [19]:
from keras.layers import *
model = Sequential()
model.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(LSTM(units=128, activation='sigmoid', recurrent_activation='hard_sigmoid', return_sequences=True))
model.add(LSTM(units=64, activation='sigmoid', recurrent_activation='hard_sigmoid', return_sequences=True))
model.add(LSTM(units=32, activation='sigmoid', recurrent_activation='hard_sigmoid', return_sequences=False))
# model.add(LSTM(units=32))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 10, 128)           2149376   
_________________________________________________________________
lstm_15 (LSTM)               (None, 10, 64)            49408     
_________________________________________________________________
lstm_16 (LSTM)               (None, 32)                12416     
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 2,211,233
Trainable params: 2,211,233
Non-trainable params: 0
_________________________________________________________________


In [20]:
estimator = []
for i in range(1,26):
    LSTM_Predictors = KerasClassifier(build_fn=lambda:model, epochs=20, batch_size=256)
    LSTM_Predictors._estimator_type="classifier"
    estimator.append((f'model{i}', LSTM_Predictors))
print(estimator)    

# estimator = [('model1', LSTM_Predictors), ('model2', LSTM_Predictors1)]

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

[('model1', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eeca0>), ('model2', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eecd0>), ('model3', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eeac0>), ('model4', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eebe0>), ('model5', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eeb80>), ('model6', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eebb0>), ('model7', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eeb20>), ('model8', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eeaf0>), ('model9', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eea90>), ('model10', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eea60>), ('model11', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eea00>), ('model12', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec

25/25 [==============================] - 2s 78ms/step - loss: 0.0137 - acc: 0.9897
Epoch 12/20
25/25 [==============================] - 2s 77ms/step - loss: 0.0130 - acc: 0.9904
Epoch 13/20
25/25 [==============================] - 2s 78ms/step - loss: 0.0131 - acc: 0.9907
Epoch 14/20
25/25 [==============================] - 2s 78ms/step - loss: 0.0133 - acc: 0.9902
Epoch 15/20
25/25 [==============================] - 2s 78ms/step - loss: 0.0123 - acc: 0.9917
Epoch 16/20
25/25 [==============================] - 2s 77ms/step - loss: 0.0120 - acc: 0.9921
Epoch 17/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0119 - acc: 0.9920
Epoch 18/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0117 - acc: 0.9923
Epoch 19/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0116 - acc: 0.9920
Epoch 20/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0116 - acc: 0.9925
Epoch 1/20
25/25 [==============================] - 2s 76ms/st

25/25 [==============================] - 2s 78ms/step - loss: 0.0094 - acc: 0.9954
Epoch 4/20
25/25 [==============================] - 2s 78ms/step - loss: 0.0088 - acc: 0.9957
Epoch 5/20
25/25 [==============================] - 2s 77ms/step - loss: 0.0086 - acc: 0.9958
Epoch 6/20
25/25 [==============================] - 2s 80ms/step - loss: 0.0089 - acc: 0.9950
Epoch 7/20
25/25 [==============================] - 2s 79ms/step - loss: 0.0085 - acc: 0.9960
Epoch 8/20
25/25 [==============================] - 2s 79ms/step - loss: 0.0091 - acc: 0.9950
Epoch 9/20
25/25 [==============================] - 2s 79ms/step - loss: 0.0084 - acc: 0.9962
Epoch 10/20
25/25 [==============================] - 2s 79ms/step - loss: 0.0090 - acc: 0.9955
Epoch 11/20
25/25 [==============================] - 2s 79ms/step - loss: 0.0086 - acc: 0.9962
Epoch 12/20
25/25 [==============================] - 2s 78ms/step - loss: 0.0088 - acc: 0.9958
Epoch 13/20
25/25 [==============================] - 2s 79ms/step - 

25/25 [==============================] - 2s 77ms/step - loss: 0.0080 - acc: 0.9970
Epoch 16/20
25/25 [==============================] - 2s 77ms/step - loss: 0.0080 - acc: 0.9966
Epoch 17/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0078 - acc: 0.9970
Epoch 18/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0083 - acc: 0.9963
Epoch 19/20
25/25 [==============================] - 2s 75ms/step - loss: 0.0080 - acc: 0.9968
Epoch 20/20
25/25 [==============================] - 2s 76ms/step - loss: 0.0078 - acc: 0.9970


VotingClassifier(estimators=[('model1',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eeca0>),
                             ('model2',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eecd0>),
                             ('model3',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eeac0>),
                             ('model4',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eebe0>),
                             ('mod...
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6ee7c0>),
                             ('model22',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6ee7f0>),
                             ('model23',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6ee6a0>),

In [21]:
preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print(f"MAE : {MAE(y_test, preds)}, MSE : {MSE(y_test, preds)}, RMSE : {RMSE(y_test, preds)}")

정확도 : 0.7185897435897436
MAE : 2.81e-01, MSE : 2.81e-01, RMSE : 5.30e-01


- 15개 <br>
정확도 : 0.7237179487179487 <br>
MAE : 2.76e-01, MSE : 2.76e-01, RMSE : 5.26e-01
- 25개 <br>
정확도 : 0.7141025641025641<br>
MAE : 2.86e-01, MSE : 2.86e-01, RMSE : 5.35e-01

In [14]:
preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print(f"MAE : {MAE(y_test, preds)}, MSE : {MSE(y_test, preds)}, RMSE : {RMSE(y_test, preds)}")

정확도 : 0.7185897435897436
MAE : 2.81e-01, MSE : 2.81e-01, RMSE : 5.30e-01


# GRU & LSTM
- 각 1개씩<br>
정확도 : 0.735897435897436<br>
MAE : 2.64e-01, MSE : 2.64e-01, RMSE : 5.14e-01
- 각 2개씩<br>
정확도 : 0.7307692307692307<br>
MAE : 2.69e-01, MSE : 2.69e-01, RMSE : 5.19e-01

In [22]:
gru = Sequential()
gru.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
gru.add(GRU(units=128, activation='sigmoid', return_sequences=True))
gru.add(GRU(units=128, activation='sigmoid'))
gru.add(Dropout(0.5))
gru.add(Dense(units=1))
gru.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])

lstm = Sequential()
lstm.add(InputLayer(input_shape=(X_train.shape[1],X_train.shape[2])))
lstm.add(LSTM(units=128, activation='sigmoid', recurrent_activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=64, activation='sigmoid', recurrent_activation='hard_sigmoid', return_sequences=True))
lstm.add(LSTM(units=32, activation='sigmoid', recurrent_activation='hard_sigmoid', return_sequences=False))
lstm.add(Dropout(0.2))
lstm.add(Dense(units=1))
lstm.compile(loss='mean_squared_error', optimizer='adam', metrics = ['acc'])

In [23]:
estimator = []
for i in range(1,6):
    lstm_predictor = KerasClassifier(build_fn=lambda:lstm, epochs=20, batch_size=256)
    lstm_predictor._estimator_type="classifier"
    gru_predictor = KerasClassifier(build_fn=lambda:gru, epochs=20, batch_size=256)
    gru_predictor._estimator_type="classifier"
    estimator.append((f'lstm{i}', lstm_predictor))
    estimator.append((f'gru{i}', gru_predictor))
print(estimator)

final_model = VotingClassifier(estimators = estimator, voting = 'soft')
final_model.fit(X_train,y_train)

[('lstm1', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eef70>), ('gru1', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9adf0>), ('lstm2', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9a7c0>), ('gru2', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9a4c0>), ('lstm3', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9ae80>), ('gru3', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9aca0>), ('lstm4', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9ac70>), ('gru4', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9aaf0>), ('lstm5', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9a580>), ('gru5', <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9a190>)]
Epoch 1/20
25/25 [==============================] - 6s 77ms/step - loss: 0.4110 - acc: 0.5126
Epoch 2/20
25/25 [==============================] - 2s 78ms/step - loss: 0.2843 

25/25 [==============================] - 2s 79ms/step - loss: 0.0199 - acc: 0.9796
Epoch 5/20
25/25 [==============================] - 2s 78ms/step - loss: 0.0171 - acc: 0.9841
Epoch 6/20
25/25 [==============================] - 2s 78ms/step - loss: 0.0172 - acc: 0.9841
Epoch 7/20
25/25 [==============================] - 2s 77ms/step - loss: 0.0164 - acc: 0.9848
Epoch 8/20
25/25 [==============================] - 2s 79ms/step - loss: 0.0155 - acc: 0.9867
Epoch 9/20
25/25 [==============================] - 2s 80ms/step - loss: 0.0150 - acc: 0.9853
Epoch 10/20
25/25 [==============================] - 2s 79ms/step - loss: 0.0168 - acc: 0.9849
Epoch 11/20
25/25 [==============================] - 2s 79ms/step - loss: 0.0151 - acc: 0.9869
Epoch 12/20
25/25 [==============================] - 2s 79ms/step - loss: 0.0151 - acc: 0.9869
Epoch 13/20
25/25 [==============================] - 2s 80ms/step - loss: 0.0151 - acc: 0.9859
Epoch 14/20
25/25 [==============================] - 2s 78ms/step -

VotingClassifier(estimators=[('lstm1',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7fec9c6eef70>),
                             ('gru1',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9adf0>),
                             ('lstm2',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9a7c0>),
                             ('gru2',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9a4c0>),
                             ('lstm3',
                              <k...
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9aca0>),
                             ('lstm4',
                              <keras.wrappers.scikit_learn.KerasClassifier object at 0x7ff0dba9ac70>),
                             ('gru4',
                              <keras.wrappers.scikit_learn.KerasClassifier o

In [24]:
preds = final_model.predict(X_test)

preds[preds>0.5]=1
preds[preds<=0.5]=0

from sklearn import metrics 
print('정확도 :', metrics.accuracy_score(y_test, preds))
print(f"MAE : {MAE(y_test, preds)}, MSE : {MSE(y_test, preds)}, RMSE : {RMSE(y_test, preds)}")

정확도 : 0.708974358974359
MAE : 2.91e-01, MSE : 2.91e-01, RMSE : 5.39e-01
